In [8]:
# load data
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Model
from keras.layers import Flatten, Dense, Input
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras import regularizers


training_path = "../Test_data/train.json"
testing_path = "../Test_data/test.json"

train_data = pd.read_json(training_path)
test_data = pd.read_json(testing_path)


#Generate the training data
#Create 2 bands having HH, HV and normalize both
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_data["band_1"]])
X_band_1 = (X_band_1-(-22.159))/(28.424**(1/2.0))
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_data["band_2"]])
X_band_2 = (X_band_2-(-24.953))/(20.669**(1/2.0))

X = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],
                          ((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

# X = np.concatenate([X_band_1[:,:,:,np.newaxis], X_band_2[:, :, :, np.newaxis]], axis=-1)



Y = train_data["is_iceberg"]
# Y = np_utils.to_categorical(y, 2)
X_full = X
Y_full = Y

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state=42)

print("size x_train", len(X_train))
print("size x_test", len(X_test))

# create the model
img_input = Input(shape=(75,75,3))
x = Convolution2D(64, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block1_conv1')(img_input)
x = Convolution2D(64, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
x = BatchNormalization()(x)

x = Convolution2D(128, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block2_conv1')(x)
x = Convolution2D(128, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
x = BatchNormalization()(x)

x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block3_conv1')(x)
x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block3_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
x = BatchNormalization()(x)

x = Flatten(name='flatten')(x)
x = Dense(512, kernel_regularizer=regularizers.l2(0.1),
          bias_regularizer=regularizers.l2(0.1))(x)
x = Dropout(0.5)(x)
x = Dense(512, kernel_regularizer=regularizers.l2(0.1),
          bias_regularizer=regularizers.l2(0.1))(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid', name='predictions')(x)

model = Model(img_input, x, name='vggBase')
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model.summary()


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = "model_weights_normed.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=15)

size x_train 1203
size x_test 401
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 37, 37, 64)        256       
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 37, 37

In [9]:
# training code
#%% Image data augmentation 
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,               # set input mean to 0 over the dataset
    samplewise_center=False,                # set each sample mean to 0
    featurewise_std_normalization=False,    # divide inputs by std of the dataset
    samplewise_std_normalization=False,     # divide each input by its std
    zca_whitening=False,                    # apply ZCA whitening
    rotation_range=10,                      # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,                  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,                 # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,                   # randomly flip images
    vertical_flip=True)                     # randomly flip images

datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, shuffle=True,
                    batch_size=32), epochs = 180,
                     verbose= 1, validation_data = (X_test, y_test),
                        callbacks=callbacks)   # validation_data = (X_test, y_test)

Epoch 1/180
38/38 [==============================] - 9s 239ms/step - loss: 222.4883 - acc: 0.5890 - val_loss: 220.8050 - val_acc: 0.4888
Epoch 2/180
38/38 [==============================] - 6s 166ms/step - loss: 220.3253 - acc: 0.5736 - val_loss: 219.1075 - val_acc: 0.4888
Epoch 3/180
38/38 [==============================] - 6s 165ms/step - loss: 218.1493 - acc: 0.5857 - val_loss: 217.6765 - val_acc: 0.4888
Epoch 4/180
38/38 [==============================] - 6s 166ms/step - loss: 215.9624 - acc: 0.6063 - val_loss: 215.8793 - val_acc: 0.4888
Epoch 5/180
38/38 [==============================] - 6s 166ms/step - loss: 213.7192 - acc: 0.6493 - val_loss: 213.8377 - val_acc: 0.4888
Epoch 6/180
38/38 [==============================] - 6s 166ms/step - loss: 211.6591 - acc: 0.6405 - val_loss: 212.2558 - val_acc: 0.4888
Epoch 7/180
38/38 [==============================] - 6s 167ms/step - loss: 209.4383 - acc: 0.6586 - val_loss: 210.0680 - val_acc: 0.4888
Epoch 8/180
38/38 [======================

38/38 [==============================] - 6s 169ms/step - loss: 112.3239 - acc: 0.8322 - val_loss: 111.5553 - val_acc: 0.8579
Epoch 61/180
38/38 [==============================] - 6s 168ms/step - loss: 110.8265 - acc: 0.8399 - val_loss: 110.1081 - val_acc: 0.8479
Epoch 62/180
38/38 [==============================] - 6s 169ms/step - loss: 109.3523 - acc: 0.8484 - val_loss: 108.6089 - val_acc: 0.8454
Epoch 63/180
38/38 [==============================] - 6s 169ms/step - loss: 107.9175 - acc: 0.8404 - val_loss: 107.1365 - val_acc: 0.8678
Epoch 64/180
38/38 [==============================] - 6s 168ms/step - loss: 106.4260 - acc: 0.8634 - val_loss: 105.7106 - val_acc: 0.8628
Epoch 65/180
38/38 [==============================] - 6s 169ms/step - loss: 105.0420 - acc: 0.8388 - val_loss: 104.2930 - val_acc: 0.8603
Epoch 66/180
38/38 [==============================] - 7s 173ms/step - loss: 103.6131 - acc: 0.8410 - val_loss: 102.8909 - val_acc: 0.8728
Epoch 67/180
38/38 [===========================

38/38 [==============================] - 6s 168ms/step - loss: 47.0291 - acc: 0.8900 - val_loss: 46.7149 - val_acc: 0.8828
Epoch 121/180
38/38 [==============================] - 6s 168ms/step - loss: 46.2868 - acc: 0.8955 - val_loss: 46.0018 - val_acc: 0.8828
Epoch 122/180
38/38 [==============================] - 6s 168ms/step - loss: 45.5886 - acc: 0.9076 - val_loss: 45.3788 - val_acc: 0.8479
Epoch 123/180
38/38 [==============================] - 6s 169ms/step - loss: 44.8912 - acc: 0.9051 - val_loss: 44.6189 - val_acc: 0.8753
Epoch 124/180
38/38 [==============================] - 6s 169ms/step - loss: 44.2109 - acc: 0.9029 - val_loss: 43.9392 - val_acc: 0.8853
Epoch 125/180
38/38 [==============================] - 6s 169ms/step - loss: 43.5398 - acc: 0.8972 - val_loss: 43.2663 - val_acc: 0.8828
Epoch 126/180
38/38 [==============================] - 6s 169ms/step - loss: 42.8571 - acc: 0.9090 - val_loss: 42.5987 - val_acc: 0.8828
Epoch 127/180
38/38 [==============================] - 

Epoch 180/180
38/38 [==============================] - 6s 168ms/step - loss: 18.9501 - acc: 0.9038 - val_loss: 18.8957 - val_acc: 0.8828


In [11]:
# from above, need to see the amount of epochs (80 +) that val stops increasing at, then do 
# that in one full go and save the model

from keras.models import load_model
import pandas as pd
# if not using val
#file_path_use = filepath_full
# if using val
file_path_use = file_path
inf_model = load_model(file_path_use)

#score = inf_model.evaluate(X_test, y_test, verbose=1)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])


X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_data["band_1"]])
X_band_test_1 = (X_band_test_1-(-22.159))/(28.424**(1/2.0))
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_data["band_2"]])
X_band_test_2 = (X_band_test_2-(-24.953))/(20.669**(1/2.0))
X_sub = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)
predicted_test=inf_model.predict(X_sub)

print("len of pred test", len(predicted_test))
print("len of id", len(test_data['id']))

submission = pd.DataFrame()
submission['id']=test_data['id']
submission['is_iceberg']=predicted_test
submission.to_csv('sub_full_normed.csv', index=False)

len of pred test 8424
len of id 8424
